# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [1]:
# Import this library
from nbacktest import Backtest, Strategy

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <b> Create Strategy </b>

In [2]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.balance/2)/self.prices["AAPL"])
        quantity2 = int((self.balance/2)/self.prices["GOOG"])

        order1 = self.buy("AAPL", quantity1)
        order2 = self.buy("GOOG", quantity2)

        trade1 = self.create_trade(orders=[order1, order2], notes="Buy and Hold Trade")

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download OHLC data from Yahoo Finance </b>

In [3]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  129.900284  89.199585  130.451959  89.487615  127.713276   
2022-12-27  128.097488  87.332367  129.456984  88.891696  126.806960   
2022-12-28  124.166794  85.872353  129.082630  87.918349  123.999322   
2022-12-29  127.683708  88.345436  128.540773  88.757616  125.831652   
2022-12-30  127.998962  88.126930  128.018669  88.226249  125.536125   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  87.024475  128.974252  87.024475  63814900  17815000  
2022-12-27  86.940055  129.427431  88.702985  69007800  15470900  
2022-12-28  85.782968  127.742842  86.905285  85438400  17879600  
2022-12-29  86.398758  126.087782  86.438487  75703700  18280700  
2022-12-30  86.438480  126.501564  86.771202  77034200  19190300

## <b> Run Backtest </b>

In [4]:
bt = Backtest(data=df_ohlc,
              universe=universe,
              strategy_class=TestStrategy,
              price_column="Close",
              cash=100_000
             )

result = bt.run()

Strategy started!
Bought 1239 AAPL and 945 GOOG
Strategy finished!


In [5]:
result.head()

,ITERATION,BALANCE,POSITIONS_TOTAL,EQUITY
0,2018-01-02,37.166557,99962.833443,100000.000000
1,2018-01-03,37.166557,100774.445103,100811.611660
2,2018-01-04,37.166557,101190.530754,101227.697311
3,2018-01-05,37.166557,102505.049973,102542.216530
4,2018-01-08,37.166557,102537.462170,102574.628727


## <b> Backtest Statistics </b>

In [6]:
print("---------------------------")
print("Final balance: %.2f" % bt.strategy.balance)
print("Final equity: %.2f" % bt.strategy.equity)
print("---------------------------")

---------------------------
Final balance: 37.17
Final equity: 241907.83
---------------------------


In [7]:
bt.statistics()

AttributeError: module 'statsmodels' has no attribute 'robust'

In [8]:
bt.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_ITERATION,REASON_CLOSED,POSITIONS_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE
0,4f0b5eef-0e28-4c28-8efa-069cbc81b8fd,OPEN,Buy and Hold Trade,141907.838379,2018-01-02,None,None,241870.66349,None,None,None


In [9]:
bt.orderbook

,ID,TICKER,ACTION,STATUS,TRADE_ID,REQUESTED_ITERATION,REQUESTED_QUANTITY,REQUESTED_PRICE,REQUESTED_FEE,REQUESTED_GROSS_TOTAL,REQUESTED_TOTAL,FILLED_ITERATION,FILLED_QUANTITY,FILLED_PRICE,FILLED_FEE,FILLED_GROSS_TOTAL,FILLED_TOTAL
0,954a51b3-20c0-4a43-ac55-1953d01c335c,AAPL,BUY,FILLED,17e63756-848b-47e9-b2d0-1745dac00e09,2018-01-02,1239,40.341881,0.0,49983.590309,49983.590309,2018-01-02,1239,40.341881,0.0,-49983.590309,-49983.590309
1,729099be-e8ef-40df-ac9c-be0cbe6d35be,GOOG,BUY,FILLED,17e63756-848b-47e9-b2d0-1745dac00e09,2018-01-02,945,52.888073,0.0,49979.228954,49979.228954,2018-01-02,945,52.888073,0.0,-49979.228954,-49979.228954


In [14]:
bt.strategy.positions

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.7522277832)},
 'GOOG': {'quantity': 945, 'value': np.float64(83279.9490737915)}}

In [16]:
!pip uninstall nbacktest --yes

Found existing installation: nbacktest 1.0.0
Uninstalling nbacktest-1.0.0:
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/lib/python3.12/shutil.py", line 886, in move
    os.rename(src, real_dst)
PermissionError: [Errno 13] Permission denied: '/srv/venv/lib/python3.12/site-packages/nbacktest-1.0.0.dist-info/' -> '/tmp/pip-uninstall-vtvcn6ze'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/venv/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 107, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "/srv/venv/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 98, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/venv/lib/python3.12/site-packages/pip/_internal/commands/uninstall.py", line 105, in run
    uninstall_pathset = req.uninstall(
                        ^^^^^^^^^^^^^^
  File "/srv/ve

In [17]:
!pip install git+https://github.com/ninebel/nbacktest.git@main

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/ninebel/nbacktest.git (to revision main) to /tmp/pip-req-build-bjdyob7j
  Running command git clone --filter=blob:none --quiet https://github.com/ninebel/nbacktest.git /tmp/pip-req-build-bjdyob7j
  Resolved https://github.com/ninebel/nbacktest.git to commit d8d82aadfcab7c8f00e12bfe8ccdb7336c23c28a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
ERROR: Will not install to the user site because it will lack sys.path precedence to nbacktest in /srv/venv/lib/python3.12/site-packages
